In [1]:
# IMPORTS 
import pandas as pd
import numpy as np
import shutil
import os
import time
import subprocess
import sys
import datetime
import matplotlib.pyplot as plt
from os.path import exists as file_exists
import matplotlib.dates as mdates

In [2]:
# FILE PATHS
data_file_path = r"C:\Users\govertsen.k\Desktop\trial\actual"
data_file_path = data_file_path.replace(os.sep, '/')
data_file_path

'C:/Users/govertsen.k/Desktop/trial/actual'

In [18]:
# CONSTANTS
study = ["baseline","outage"] #baseline
year =  "2001" #
otg_date = "August 10" # start outage on this day
otg_hour = ["8","12","20"] # start outage just after this hour
otg_len = "6" # approximate hours

sample_dates = [['08/09 00:00:00'],['08/12 23:00:00']]
index = ['start','end']
columns = ['Date/Time']
sample = pd.DataFrame(sample_dates,index,columns)
sample['Date/Time']= pd.to_datetime(sample['Date/Time'], format = '%m/%d %H:%M:%S')
   

# Identify all the models
models = [os.path.splitext(filename)[0] for filename in os.listdir(data_file_path + '/output/' + study[0] + "/" + year)]

In [19]:
# - Temperature
baseline_temperature = pd.DataFrame(columns = models)
# - Electricity
baseline_electricity =pd.DataFrame(columns = models)
# - Gas 
baseline_gas = pd.DataFrame(columns = models)
# - Humidity
baseline_humidity = pd.DataFrame(columns = models)


In [20]:
# Make new summary folders
new_path = data_file_path + "/" + "summary"
if not os.path.exists(new_path):
    os.makedirs(new_path)
for each_study in study:
    newer_path = new_path + "/" + year
    if not os.path.exists(newer_path):
        os.makedirs(newer_path)

In [21]:
# Save Outdoor Data
outdoor = pd.DataFrame()
data = pd.read_csv(data_file_path + '/output/baseline/' + year + "/" + models[0] + '.csv')
data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
outdoor['Date/Time'] = data['Date/Time']
outdoor['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']

# Change '24:00:00' to "00:00:00", set datetime, add one day to midnight values, reset index, and format
outdoor['Date/Time'] = outdoor['Date/Time'].str.strip()
outdoor['Date/Time'] = outdoor['Date/Time'].str.replace('24:00:00','00:00:00')
outdoor['Date/Time'] = pd.to_datetime(outdoor['Date/Time'],format = '%m/%d  %H:%M:%S')
outdoor.loc[outdoor['Date/Time'].dt.hour == 0,'Date/Time'] = outdoor.loc[outdoor['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')
outdoor = outdoor.reset_index(drop=True)

# Trim down just to sample dates
outdoor = outdoor.loc[(outdoor['Date/Time'] >= sample['Date/Time'][0]) & (outdoor['Date/Time'] <= sample['Date/Time'][1])]
outdoor = outdoor.reset_index(drop=True)
    
outdoor.to_csv(data_file_path + "/summary/" + year + "/outdoor.csv",index=False)

In [22]:
# BASELINES

# - Temperature
baseline_temperature = pd.DataFrame(columns = models)

# - Humidity
baseline_humidity = pd.DataFrame(columns = models)

# - Electricity
baseline_electricity =pd.DataFrame(columns = models)
# - Gas 
baseline_gas = pd.DataFrame(columns = models)

for each_model in models:
    ## Baseline
    # Create empty dataframe
    baseline = pd.DataFrame()

    # Read the baseline output file
    file_path = data_file_path + '/output/baseline/' + year + "/" + each_model + ".csv"
    data = pd.read_csv(file_path)
    data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
    baseline['Date/Time'] = data['Date/Time']
    baseline['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']
    baseline['RH'] = data['LIVING ZONE:Zone Air Relative Humidity [%](Hourly)']
    baseline['Indoor Temp'] = data['LIVING ZONE:Zone Air Temperature [C](Hourly)']
    baseline['Electricity'] = data['Electricity:Facility [J](Hourly)']
    if 'NaturalGas:Facility [J](TimeStep) ' in data:
        baseline['Gas'] = data['NaturalGas:Facility [J](TimeStep) ']

    # Change '24:00:00' to "00:00:00", set datetime, add one day to midnight values, reset index, and format
    baseline['Date/Time'] = baseline['Date/Time'].str.strip()
    baseline['Date/Time'] = baseline['Date/Time'].str.replace('24:00:00','00:00:00')
    baseline['Date/Time'] = pd.to_datetime(baseline['Date/Time'],format = '%m/%d  %H:%M:%S')
    baseline.loc[baseline['Date/Time'].dt.hour == 0,'Date/Time'] = baseline.loc[baseline['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')
    baseline = baseline.reset_index(drop=True)

    # Trim down just to sample dates
    baseline = baseline.loc[(baseline['Date/Time'] >= sample['Date/Time'][0]) & (baseline['Date/Time'] <= sample['Date/Time'][1])]
    baseline = baseline.reset_index(drop=True)
    
    # Transfer to consilidated list 
    baseline_temperature[each_model] = baseline["Indoor Temp"]
    baseline_humidity[each_model] = baseline["RH"]
    baseline_electricity[each_model] = baseline["Electricity"]
    if 'Gas' in baseline:
        baseline_gas[each_model] = baseline["Gas"]

# Save as CSV
baseline_temperature = baseline_temperature.set_index(baseline['Date/Time'])
baseline_humidity = baseline_humidity.set_index(baseline['Date/Time'])
baseline_electricity = baseline_electricity.set_index(baseline['Date/Time'])
baseline_gas = baseline_gas.set_index(baseline['Date/Time'])
baseline_temperature.to_csv(data_file_path + "/summary/" + year + "/baseline_temperature.csv",index=True)
baseline_humidity.to_csv(data_file_path + "/summary/" + year + "/baseline_humidity.csv",index=True)
baseline_electricity.to_csv(data_file_path + "/summary/" + year + "/baseline_electricity.csv",index=True)
baseline_gas.to_csv(data_file_path + "/summary/" + year + "/baseline_gas.csv",index=True)
baseline = baseline.reset_index(drop=True)
baseline_temperature = baseline_temperature.reset_index(drop=True)

In [23]:
baseline_temperature

,bldg0002642,bldg0003104,bldg0006733,bldg0009153,bldg0010160,bldg0010982,bldg0012954,bldg0013107,bldg0013299,bldg0014965,...,bldg0527700,bldg0528109,bldg0533187,bldg0534041,bldg0535065,bldg0536627,bldg0539572,bldg0539770,bldg0544793,bldg0547838
0,23.888905,25.555558,25.555558,22.218452,23.888893,30.894492,21.106615,35.964584,21.847605,22.052199,...,35.900315,23.888927,23.352307,22.417313,21.350334,25.555555,40.619847,26.661968,18.995830,22.818200
1,23.888907,24.465051,25.555557,22.218794,23.888893,29.734364,21.107197,35.636322,21.635663,21.623541,...,36.230018,23.888923,23.357289,22.457857,21.335203,25.555716,41.193150,26.592053,19.200938,22.786574
2,23.888908,24.444448,25.555557,22.219124,23.888893,28.625044,21.107666,35.219246,21.427481,21.214345,...,34.788471,23.888918,23.053655,20.898349,21.111224,25.544267,39.667156,25.799633,18.884434,21.578776
3,23.888908,24.444448,25.512726,22.219438,23.888892,27.580367,21.108110,34.759992,21.207736,20.810888,...,35.014884,23.888921,23.084857,20.887111,21.130420,25.469571,40.618683,26.238858,18.942683,21.605792
4,23.888906,24.444448,25.291956,22.219749,23.888890,26.606146,21.108510,34.325105,21.112065,20.409062,...,33.529177,23.888912,23.054298,20.023656,21.111114,25.192230,39.372143,25.558770,18.885341,23.397934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,23.729934,24.666324,23.963868,22.221888,23.225258,24.881425,21.109264,29.859337,21.111124,19.997540,...,27.218202,23.888905,23.054477,19.995157,21.111114,24.393569,34.776126,24.106205,18.886989,22.563080
92,23.736304,24.393720,23.794150,22.220760,23.242449,24.542430,21.108153,29.884867,21.170126,19.997326,...,27.052199,23.888909,23.054691,19.995159,21.111115,24.312716,34.674213,23.839347,18.887007,21.180411
93,23.630264,24.638634,23.752682,22.221758,23.607517,24.692004,21.109249,30.380774,21.113857,19.996814,...,27.076530,23.888921,23.054383,19.995214,21.111115,24.096058,34.046400,23.515646,18.886618,20.829379
94,23.793672,24.726534,23.637628,22.221001,23.753355,24.468390,21.107043,30.617268,21.230086,19.996319,...,27.159634,23.888918,23.053803,20.088762,21.111331,24.036344,34.855400,23.825017,18.885699,20.968591


In [24]:
for each_otg_start in otg_hour:
    # OUTAGE
    # - Temperature
    outage_temperature = pd.DataFrame(columns = models)
    
    # - Humidity
    outage_humidity = pd.DataFrame(columns = models)
    
    # - Temperature Difference
    outage_temp_diff = pd.DataFrame(columns = models)
    
    # - Electricity
    outage_electricity = pd.DataFrame(columns = models)
    
    # - Gas
    outage_gas = pd.DataFrame(columns = models)

    for each_model in models: 

        ## Outage
        # Create empty dataframe
        outage = pd.DataFrame()

        data = pd.read_csv(data_file_path + '/output/outage/' + year + "/" + each_otg_start +  "/" + each_model + '.csv')
        data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
        outage['Date/Time'] = data['Date/Time']
        outage['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']
        outage['Indoor Temp'] = data['LIVING ZONE:Zone Air Temperature [C](Hourly)']
        outage['RH'] = data['LIVING ZONE:Zone Air Relative Humidity [%](Hourly)']
        outage['Electricity'] = data['Electricity:Facility [J](Hourly)']
        if 'NaturalGas:Facility [J](TimeStep) ' in data:
            outage['Gas'] = data['NaturalGas:Facility [J](TimeStep) ']

        outage['Date/Time'] = outage['Date/Time'].str.strip()
        outage['Date/Time'] = outage['Date/Time'].str.replace('24:00:00','00:00:00')
        outage['Date/Time'] = pd.to_datetime(outage['Date/Time'],format = '%m/%d  %H:%M:%S')

        outage.loc[outage['Date/Time'].dt.hour == 0,'Date/Time'] = outage.loc[outage['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')

        outage=outage.reset_index(drop=True)

        # Reduce to outage date
        outage = outage.loc[(outage['Date/Time'] >= sample['Date/Time'][0]) & (outage['Date/Time'] <= sample['Date/Time'][1])]
        outage = outage.reset_index(drop=True)
        outage_temperature[each_model] = outage['Indoor Temp']
        outage_humidity[each_model] = outage['RH']
        outage_electricity[each_model] = outage["Electricity"]
        outage_temp_diff[each_model] = outage['Indoor Temp'] - baseline_temperature[each_model]

        if 'Gas' in outage.columns:
            outage_gas[each_model] = outage["Gas"]
        outage = outage.set_index(outage['Date/Time'])

    outage_temperature = outage_temperature.set_index(outage['Date/Time'])
    outage_electricity = outage_electricity.set_index(outage['Date/Time'])
    outage_gas =  outage_gas.set_index(outage['Date/Time'])
    outage_temp_diff = outage_temp_diff.set_index(outage['Date/Time'])
    outage_humidity = outage_humidity.set_index(outage['Date/Time'])
   

    outage_temperature.to_csv(data_file_path + "/summary/" + year + "/outage_temperature_"+ each_otg_start + "_start.csv",index=True)
    outage_electricity.to_csv(data_file_path + "/summary/" + year + "/outage_electricity_" + each_otg_start + "_start.csv",index=True)
    outage_gas.to_csv(data_file_path + "/summary/" + year + "/outage_gas_"+ each_otg_start +"_start.csv",index=True)
    outage_temp_diff.to_csv(data_file_path + "/summary/" + year + "/outage_temp_diff_"+ each_otg_start + "_start.csv",index=True)
    outage_humidity.to_csv(data_file_path + "/summary/" + year + "/outage_humidity_"+ each_otg_start + "_start.csv",index=True)
  